### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv("../Starter_Code/M6_Starter_Code/.env")

#setting the api keys to variables
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
#test prints to make sure everything is working as intended
#print(nyt_api_key)
#print(tmdb_api_key)

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
base_query_url=f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
base_query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=6gZp08FtAh3U8Bjh7UsgWa08aYnhWpjm&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531'

In [4]:
# Create an empty list to store the reviews
reviews_list=[]

# loop through pages 0-19
for page in range(0,20):
    # create query with a page number
    query_url=f"{base_query_url}&page={str(page)}"
    # API results show 10 articles at a time
    print(query_url)
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    print(response)
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    if response.status_code == 200:
        try:
            articles=response.json()
            print(articles)
            if articles["response"]["docs"]:
                
                for article in articles["response"]["docs"]:
                    # loop through the reviews["response"]["docs"] and append each review to the list
                    reviews_list.append(article)
                    # Print the page that was just retrieved
                    print(f"Page {page} retrieved.")
            else:
                    # Print the page number that had no results then break from the loop
                print(f"Page {page} has no results.")
                break
        except Exception as e:
            print(f"JSON error: {e}")
            
    else:    
        print(f"error on page {page}: {response.status_code}")


https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=6gZp08FtAh3U8Bjh7UsgWa08aYnhWpjm&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&page=0
<Response [200]>
{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': '

In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
articles5=articles["response"]["docs"][:5]
print(json.dumps(articles5, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2017/05/11/movies/hounds-of-love-review.html",
        "snippet": "This feature debut of the Australian filmmaker Ben Young resembles a real-life couple\u2019s killing spree in the 1980s.",
        "source": "The New York Times",
        "headline": {
            "main": "Review: \u2018Hounds of Love\u2019 Is Tense and Deadly Down Under",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Hounds of Love",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Hounds of Love (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
     

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
from pandas import json_normalize
reviews_df = json_normalize(reviews_list)
reviews_df.head(2)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

#I dont like lambda so i did it this way.
#defining a function that will extract the title from the headline.main col
def extract_func(headline):
    #checking that the unicode characters are found in the headline.main col
    if '\u2018' in headline and '\u2019' in headline:
        """ Extracts the title from the headline text between 
        unicode characters \u2018 and \u2019.
    Variables:
        a: finds the first character after \u2018 
        b: finds the last character which is right before \u2019
        title (str): pulls the information between a and b into a string and
            adds "Review" to the end. 
    Args: 
        headline (str): The headline text.
        
    Returns:
        title (str): The extracted title or the original headline 
        if the characters are not found.
    """
        

        a = headline.find('\u2018')+1       
        b = headline.rfind('\u2019')
        title = headline[a:b]
        
    # if there are no unicode characters, it will return the 
    # headline.main as the title    
    else:
        title = headline
        
    #returns title for use in creating a new column
    return title

#adding the column title to the dataframe and applying the extract function to
#the headline.main col
reviews_df["title"]=reviews_df["headline.main"].apply(extract_func)
reviews_df.head(5)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"]=reviews_df["keywords"].apply(extract_keywords)
reviews_df.head(3)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever


In [9]:
# Create a list from the "title" column using to_list()
title_list=reviews_df["title"].tolist()
title_list
# These titles will be used in the query for The Movie Database


['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate’ Review: Love the One You',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder’ Review: A God',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction’ Review: A Poet',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
print(url+tmdb_key_string)

https://api.themoviedb.org/3/search/movie?query=&api_key=28e2274d8bd40e0a2472788879c22701


In [11]:
# Create an empty list to store the results
tmbd_results_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter=0


# Loop through the titles
for t in title_list:
    # Check if we need to sleep before making a request
    if request_counter >0 and request_counter % 50 == 0:
        time.sleep(1)
        print("sleeping...")
    # Add 1 to the request counter
    request_counter+=1
    
    # Perform a "GET" request for The Movie Database
    search = f"{url}{t}{tmdb_key_string}"
    response_tmbd=requests.get(search)
    
    # checking that a response is recieved (200)
    if response_tmbd.status_code == 200:
        #converting the response info to json
        movie_data = response_tmbd.json()
        #Debugging: Print search URL and response data
        #print(f"Search URL: {search}")
        #print(f"Response Data: {movie_data}")    
    
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
        try:
            # Get movie id by pulling the id from the json dict
            movie_id = movie_data["results"][0]["id"]
            # Make a request for a the full movie details
            
            movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}'

            # Execute "GET" request with url
            movie_response=requests.get(movie_url)
            movie_info = movie_response.json()
            # Extract the genre names into a list
            genres= [genre["name"] for genre in movie_info["genres"]]

            # Extract the spoken_languages' English name into a list
            lang = [lang["english_name"] for lang in movie_info["spoken_languages"]]

            # Extract the production_countries' name into a list
            prod_country = [country["name"] for country in movie_info["production_countries"]]

            # Add the relevant data to a dictionary and
            # append it to the tmdb_movies_list list
            
            movie_dict={
                "title":t,
                "original_title":movie_info["original_title"],
                "budget":movie_info["budget"],
                "genre": genres,
                "language":movie_info["original_language"],
                "spoken_languages": lang,
                "homepage": movie_info["homepage"],
                "overview":movie_info["overview"],
                "popularity":movie_info["popularity"],
                "runtime":movie_info["runtime"],
                "revenue":movie_info["revenue"],
                "release_date":movie_info["release_date"],
                "vote_average":movie_info["vote_average"],
                "vote_count":movie_info["vote_count"],
                "production_countries":prod_country
                }
            tmbd_results_list.append(movie_dict)
            # Print out the title that was found
            print(f"Found {t}")
        except:
            #printing that no results were found if the try failed
            print(f"No results for {t}")
            
    else:
        #printing an error if the response code is incorrect or the req counter
        print(f"Error on response: {response_tmbd.status_code}, request counter: {request_counter}")

Found The Attachment Diaries
Found What’s Love Got to Do With It?
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
No results for About Fate’ Review: Love the One You
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
No results for Thor: Love and Thunder’ Review: A God
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
No results for Benediction’ Review: A Poet
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La M

In [12]:
#reviewing the list from the previous loop
tmbd_results_list

[{'title': 'The Attachment Diaries',
  'original_title': 'El apego',
  'budget': 0,
  'genre': ['Drama', 'Mystery', 'Thriller', 'Horror'],
  'language': 'es',
  'spoken_languages': ['Spanish'],
  'homepage': '',
  'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.',
  'popularity': 1.975,
  'runtime': 102,
  'revenue': 0,
  'release_date': '2021-10-07',
  'vote_average': 3.0,
  'vote_count': 4,
  'production_countries': ['Argentina']},
 {'title': 'What’s Love Got to Do With It?',
  'original_title': "What's Love Got to Do with It?",
  'budget': 0,
  'genre': ['Romance', 'Comedy'],
  'language': 'en',
  'spoken_languages': ['English', 'Portug

In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

#making a variable to hold the first 5 values in the tmbd results list
five = tmbd_results_list[:5]
for r in five:
    print(json.dumps(r, indent=4))

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "genre": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "language": "es",
    "spoken_languages": [
        "Spanish"
    ],
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 1.975,
    "runtime": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 3.0,
    "vote_count": 4,
    "production_countries": [
        "Argentina"
    ]
}
{
    "title": "What\u2019s Love Got to Do With It?",
    "original_title": "What's Love Got to Do with It?",
 

In [14]:
# Convert the results to a DataFrame
tmbd_df=pd.DataFrame(tmbd_results_list)
tmbd_df.head(5)

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.975,102,0,2021-10-07,3.000,4,[Argentina]
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"[Romance, Comedy]",en,"[English, Portuguese, Urdu]",,Two childhood friends now in their thirties mu...,23.347,109,10898395,2023-01-26,6.074,175,"[France, United Kingdom]"
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,15055,2023-03-24,6.600,37,"[Canada, United States of America]"
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",21.182,96,0,2023-04-21,6.287,162,[United States of America]
4,Other People’s Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",15.071,104,84178,2022-09-21,6.818,190,[France]


### Merge and Clean the Data for Export

In [15]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df=pd.merge(tmbd_df,reviews_df,on="title",how="inner")
merged_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.975,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"[Romance, Comedy]",en,"[English, Portuguese, Urdu]",,Two childhood friends now in their thirties mu...,23.347,109,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",21.182,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People’s Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",15.071,104,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [16]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genre","spoken_languages","production_countries",]

# Create a list of characters to remove
char_to_remove=["[","]","'","'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[col]=merged_df[col].astype(str)

    # Loop through characters to remove
    for char in char_to_remove:
        merged_df[col]=merged_df[col].str.replace(char,"")

# Display the fixed DataFrame
merged_df.head(4)

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.975,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"Romance, Comedy",en,"English, Portuguese, Urdu",,Two childhood friends now in their thirties mu...,23.347,109,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",21.182,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None


In [17]:
# Drop "byline.person" column
merged_df.drop(columns=["byline.person"],inplace=True)
merged_df.head(4)

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.975,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"Romance, Comedy",en,"English, Portuguese, Urdu",,Two childhood friends now in their thirties mu...,23.347,109,...,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",21.182,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None


In [18]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True,inplace=True)
merged_df.head(4)

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.975,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"Romance, Comedy",en,"English, Portuguese, Urdu",,Two childhood friends now in their thirties mu...,23.347,109,...,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",21.182,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None


In [19]:
# Export data to CSV without the index
merged_df.to_csv("../Final_Code/Final_Data_Output.csv",index=False)